In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf

In [2]:
import sys
sys.path.append('../input/pretrained-weights-for-facenet/')
from  fr_utils import *
from inception_blocks_v2 import *

In [3]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [4]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
  
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1)
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

In [5]:
with tf.compat.v1.Session() as test:
    tf.random.set_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random.normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random.normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random.normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

loss = 528.1426


In [6]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [7]:
database = {}
database["danielle"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("../input/pretrained-weights-for-facenet/images/images/arnaud.jpg", FRmodel)
database["aaron_eckhart"] = img_to_encoding("/kaggle/input/masked-test/Aaron_Eckhart_0001.jpg", FRmodel)
database["Aaron_Guiel"] = img_to_encoding("/kaggle/input/masked-test/Aaron_Guiel_0001.jpg", FRmodel)
database["Aaron_Patterson"] = img_to_encoding("/kaggle/input/masked-test/Aaron_Patterson_.jpg", FRmodel)
#database["Aaron_Peirsol"] = img_to_encoding("/kaggle/input/masked-test/Aaron_Peirsol.jpg", FRmodel)
database["Aaron_Pena"] = img_to_encoding("/kaggle/input/masked-test/Aaron_Pena.jpg", FRmodel)
database["Aaron_Sorkin"] = img_to_encoding("/kaggle/input/masked-test/Aaron_Sorkin.jpg", FRmodel)
database["Aaron_Tippin"] = img_to_encoding("/kaggle/input/masked-test/Aaron_Tippin.jpg", FRmodel)
database["Elizabeth_Berkeley"] = img_to_encoding("/kaggle/input/masked-test/Elizabeth_Berkeley.jpg", FRmodel)
#database["abdullah"] = img_to_encoding("/kaggle/input/masked-test/abdullah.jpg", FRmodel)
#database["abdulrahman"] = img_to_encoding("/kaggle/input/masked-test/abdulrahman.jpg", FRmodel)

In [25]:
emb = img_to_encoding("/kaggle/input/masked-test/Aaron_Pena.jpg", FRmodel)
print(emb)

[[ 0.12482239  0.02805366 -0.08227151 -0.04046029  0.15539831  0.13699976
   0.13662408 -0.00824939 -0.10897403 -0.02378259  0.01183019  0.08370211
   0.13081479 -0.02503136  0.01467813 -0.04501245  0.02357912 -0.00808508
  -0.10171583  0.21611272  0.10171434 -0.01920632  0.04075658  0.12729177
  -0.06259141 -0.04992387 -0.13884544 -0.13491002  0.11487987  0.11882837
  -0.04177491  0.0762508  -0.07601203  0.08005761  0.00360007  0.03067155
  -0.01843933  0.15740044 -0.07656772 -0.08286919  0.02876028  0.01349174
  -0.06748188  0.01055167 -0.17158644  0.05382554  0.10494261  0.09011213
  -0.11443621  0.06718607 -0.05535607 -0.00052094 -0.03436427 -0.01176701
   0.1384948  -0.02677095 -0.07868091  0.19726107 -0.09269502 -0.07172609
  -0.10084908  0.04863691  0.2164248  -0.1529929   0.0748748   0.00086365
   0.01468171  0.05398401 -0.15222558 -0.04012606  0.03519385  0.08288232
  -0.03531674  0.058392    0.0101541   0.06016127  0.0261086   0.00692859
   0.11883429  0.04610551  0.00135621 

In [20]:
def verify(image_path, identity, database, model):
    
    encoding = img_to_encoding(image_path, model)
    dist = np.linalg.norm((encoding - database[identity]))
    if dist < 0.4:
        print("It's " + str(identity))
        door_open = True
    else:
        print("It's not " + str(identity))
        door_open = False
        
    return dist, door_open

In [23]:
verify("/kaggle/input/masked-test/abdulrahman.jpg", "aaron_eckhart", database, FRmodel)

It's not aaron_eckhart


(0.72605395, False)

In [10]:
verify("/kaggle/input/pretrained-weights-for-facenet/images/images/camera_3.jpg", "bertrand", database, FRmodel)

It's bertrand


(0.35898253, True)

In [11]:
verify("../input/pretrained-weights-for-facenet/images/images/camera_2.jpg", "kian", database, FRmodel)

It's not kian


(0.85868865, False)

In [18]:
verify("/kaggle/input/masked-test/Elizabeth_Berkeley.jpg", "Elizabeth_Berkeley", database, FRmodel)

It's Elizabeth_Berkeley


(0.0, True)

In [19]:
verify("/kaggle/input/masked-test/Aaron_Patterson_.jpg", "Elizabeth_Berkeley", database, FRmodel)

It's Elizabeth_Berkeley


(0.6725045, True)

In [29]:


def who_is_it(image_path, database, model):
    encoding = img_to_encoding(image_path, model)   
    min_dist = 100
    
    for (name, db_enc) in database.items():
        dist = np.linalg.norm((encoding - db_enc))
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.4:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [31]:
who_is_it("/kaggle/input/pretrained-weights-for-facenet/images/images/andrew.jpg", database, FRmodel)

it's andrew, the distance is 0.0


(0.0, 'andrew')